In [1]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from dotenv import load_dotenv

load_dotenv()

car_template = """You are an expert in automobiles. You have extensive knowledge about car mechanics, \
models, and automotive technology. You provide clear and helpful answers about cars.

Here is a question:
{query}"""

restaurant_template = """You are a knowledgeable food critic and restaurant reviewer. You have a deep understanding of \
different cuisines, dining experiences, and what makes a great restaurant. You answer questions about restaurants insightfully.

Here is a question:
{query}"""

technology_template = """You are a tech expert with in-depth knowledge of the latest gadgets, software, \
and technological trends. You provide insightful and detailed answers about technology.

Here is a question:
{query}"""

In [2]:
car_questions = [
    "What is the difference between a sedan and an SUV?",
    "How does a hybrid car save fuel?",
    "What should I look for when buying a used car?",
]

restaurant_questions = [
    "What makes a five-star restaurant exceptional?",
    "How do I choose a good wine pairing for my meal?",
    "What are the key elements of French cuisine?",
]

technology_questions = [
    "What are the latest advancements in AI?",
    "How do I secure my home network against cyber threats?",
    "What should I consider when buying a new smartphone?",
]

In [3]:
embeddings = OpenAIEmbeddings()

car_question_embeddings = embeddings.embed_documents(car_questions)
restaurant_question_embeddings = embeddings.embed_documents(restaurant_questions)
technology_question_embeddings = embeddings.embed_documents(technology_questions)

In [4]:
def prompt_router(input):
    query_embedding = embeddings.embed_query(input["query"])
    car_similarity = cosine_similarity([query_embedding], car_question_embeddings)[0]
    restaurant_similarity = cosine_similarity(
        [query_embedding], restaurant_question_embeddings
    )[0]
    technology_similarity = cosine_similarity(
        [query_embedding], technology_question_embeddings
    )[0]

    max_similarity = max(
        max(car_similarity), max(restaurant_similarity), max(technology_similarity)
    )

    if max_similarity == max(car_similarity):
        print("Using CAR")
        return PromptTemplate.from_template(car_template)
    elif max_similarity == max(restaurant_similarity):
        print("Using RESTAURANT")
        return PromptTemplate.from_template(restaurant_template)
    else:
        print("Using TECHNOLOGY")
        return PromptTemplate.from_template(technology_template)


input_query = {"query": "What's the best way to improve my cars's battery life?"}
prompt = prompt_router(input_query)

Using CAR


In [5]:
chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | ChatOpenAI()
    | StrOutputParser()
)

In [6]:
chain.invoke("How do I identify a good vintage wine at a restaurant?")

Using RESTAURANT


"Identifying a good vintage wine at a restaurant can be a fun and rewarding experience. Here are some tips to help you choose a great vintage wine:\n\n1. Look for reputable wine regions: Wines from well-known and respected regions such as Bordeaux, Burgundy, Napa Valley, and Tuscany are more likely to be of high quality.\n\n2. Check the wine list: A restaurant that has a well-curated wine list with a good selection of vintage wines is more likely to have a knowledgeable sommelier or wine director who can help guide you to a great choice.\n\n3. Consider the price: While price doesn't always equate to quality, vintage wines are typically more expensive due to their rarity and aging process. If a vintage wine is priced significantly lower than expected, it may not be of the best quality.\n\n4. Ask for recommendations: Don't be afraid to ask your server or sommelier for recommendations based on your preferences. They can often provide valuable insights and help you find a vintage wine that

Classification


In [7]:
classification_template = PromptTemplate.from_template(
    """You are good at classifying a question.
    Given the user question below, classify it as either being about `Car`, `Restaurant`, or `Technology`.

    <If the question is about car mechanics, models, or automotive technology, classify it as 'Car'>
    <If the question is about cuisines, dining experiences, or restaurant services, classify it as 'Restaurant'>
    <If the question is about gadgets, software, or technological trends, classify it as 'Technology'>

    <question>
    {question}
    </question>

    Classification:"""
)

classification_chain = classification_template | ChatOpenAI() | StrOutputParser()

In [8]:
def prompt_router(input):
    classification = classification_chain.invoke({"question": input["query"]})

    if classification == "Car":
        print("Using CAR")
        return PromptTemplate.from_template(car_template)
    elif classification == "Restaurant":
        print("Using RESTAURANT")
        return PromptTemplate.from_template(restaurant_template)
    elif classification == "Technology":
        print("Using TECHNOLOGY")
        return PromptTemplate.from_template(technology_template)
    else:
        print("Unexpected classification:", classification)
        return None


input_query = {"query": "What are the latest trends in electric cars?"}
prompt = prompt_router(input_query)

Using TECHNOLOGY


In [9]:
chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | ChatOpenAI()
    | StrOutputParser()
)

In [10]:
chain.invoke("How do I identify a good vintage wine at a restaurant?")

Using RESTAURANT


"Identifying a good vintage wine at a restaurant can be a bit intimidating, but there are a few key things to look for:\n\n1. Check the wine list: A reputable restaurant will have a well-curated wine list with a good selection of vintage wines. Look for well-known producers and regions known for producing high-quality wines.\n\n2. Ask the sommelier: If you're unsure about which vintage wine to choose, don't hesitate to ask the sommelier for recommendations. They are trained professionals who can guide you based on your preferences and budget.\n\n3. Look for specific years: Vintage wines are labeled with the year the grapes were harvested. In general, wines from good vintages tend to be of higher quality. Research the specific year of the wine you're interested in to see if it was a good vintage.\n\n4. Consider the region: Different wine regions have different climates and growing conditions that can affect the quality of the wine. Research the specific region of the wine to get an idea

### Tool Calling


In [11]:
from langchain_core.tools import tool


@tool
def classify_as_car(question: str) -> str:
    """Classifies the question as 'Car' if it is about car mechanics, models, or automotive technology.

    Args:
        question: The input question to classify.
    """
    print("Function 'classify_as_car' was used.")
    return "Car"


@tool
def classify_as_restaurant(question: str) -> str:
    """Classifies the question as 'Restaurant' if it is about cuisines, dining experiences, or restaurant services.

    Args:
        question: The input question to classify.
    """
    print("Function 'classify_as_restaurant' was used.")
    return "Restaurant"


@tool
def classify_as_technology(question: str) -> str:
    """Classifies the question as 'Technology' if it is about gadgets, software, or technological trends.

    Args:
        question: The input question to classify.
    """
    print("Function 'classify_as_technology' was used.")
    return "Technology"

In [12]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, ToolMessage

llm = ChatOpenAI()

query = "How do I identify a good vintage wine at a restaurant?"

tools = [classify_as_car, classify_as_restaurant, classify_as_technology]
llm_with_tools = llm.bind_tools(tools)

messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {
        "classify_as_car": classify_as_car,
        "classify_as_restaurant": classify_as_restaurant,
        "classify_as_technology": classify_as_technology,
    }[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

messages

Function 'classify_as_restaurant' was used.


[HumanMessage(content='How do I identify a good vintage wine at a restaurant?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_zQyXbCvx1YtWqDAqYxZW6rGg', 'function': {'arguments': '{"question":"How do I identify a good vintage wine at a restaurant?"}', 'name': 'classify_as_restaurant'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 188, 'total_tokens': 216, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7175a6f1-9dd2-4c43-ab76-972e191239d7-0', tool_calls=[{'name': 'classify_as_restaurant', 'args': {'question': 'How do I identify a good vintage wine at a restaurant?'}, 'id':

In [13]:
llm_with_tools.invoke(messages)

AIMessage(content="Since your question is related to identifying a good vintage wine at a restaurant, I will provide you with some tips on how to do so. Let's proceed with the information.\nTo identify a good vintage wine at a restaurant, you can follow these tips:\n\n1. **Study the Wine List**: Look for wines with older vintages as they tend to be more valuable and have better quality. Check for renowned wine regions and producers known for their vintage wines.\n\n2. **Ask the Sommelier**: The sommelier at the restaurant is an expert in wines and can guide you in selecting a good vintage wine based on your preferences and budget. They can also provide you with information about the wine's origin and quality.\n\n3. **Consider the Price**: Vintage wines are usually more expensive than non-vintage wines. While price is not always an indicator of quality, it can give you an idea of the wine's rarity and reputation.\n\n4. **Look for Signs of Aging**: Inspect the wine label and cork for sig